In [ ]:
import numpy as np 
import pandas as pd
import sklearn
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt

# grid search k value for SMOTE oversampling for imbalanced classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
maindf=pd.read_csv("../input/amazon-ml-engineer-hiring-challenge/train.csv")
maindf[0:3]

In [ ]:
print(maindf.info())
print(maindf.describe())

customer segment and X1 are categorical variables. and our target variable is a binary response ( 0 and 1)

In [ ]:
if round( (sum(maindf.isnull().sum())/len(maindf) )*100,3) <= 5: # 5% is our threshold 
    print(round( (sum(maindf.isnull().sum())/len(maindf) )*100,3),"%. Remove the NaN data")
    
else:
    print("Preprocessing required")

In [ ]:
maindf=maindf.dropna()
maindf.pop("customer_id")

In [ ]:
print(maindf["customer_active_segment"].value_counts())
print("\n")
print(maindf["X1"].value_counts())
print("\n")
print(maindf["customer_category"].value_counts())

# no.of.'0'/ no.of.'0'+ no.of.'1' %
print("% for 0",round ( ( maindf["customer_category"].value_counts()[0] /( maindf["customer_category"].value_counts()[0]+maindf["customer_category"].value_counts()[1]))*100,3))
print("% for 1",round ( (maindf["customer_category"].value_counts()[1] /( maindf["customer_category"].value_counts()[0]+maindf["customer_category"].value_counts()[1]))*100,3))

# the response variable is imbalanced.

In [ ]:
sns.pairplot(data=maindf,hue='customer_category')
# skips 'X1' and 'customer_active_segment'


In [ ]:
sns.pairplot(data=maindf, kind="kde")

In [ ]:
corr_mat = maindf.corr().stack().reset_index(name="correlation")

# Draw each cell as a scatter point with varying size and color
g = sns.relplot(
    data=corr_mat,
    x="level_0", y="level_1", hue="correlation", size="correlation",
    palette="vlag", hue_norm=(-1, 1), edgecolor=".7",
    height=10, sizes=(50, 250), size_norm=(-.2, .8),
)

# Tweak the figure to finalize
g.set(xlabel="", ylabel="", aspect="equal")
g.despine(left=True, bottom=True)
g.ax.margins(.02)
for label in g.ax.get_xticklabels():
    label.set_rotation(90)
for artist in g.legend.legendHandles:
    artist.set_edgecolor(".7")

In [ ]:
y=maindf.pop('customer_category')
x=pd.get_dummies(maindf,columns=['customer_active_segment','X1'])

print(x)
print(y)

In [ ]:

#use smoteen
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN 

print('Original dataset shape %s' % Counter(y))
sme = SMOTEENN(random_state=42)
X_res, y_res = sme.fit_resample(x, y)
print('Resampled dataset shape %s' % Counter(y_res))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.20, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
clf1=KNeighborsClassifier()
model1=clf1.fit(x_train,y_train)
pred_y=model1.predict(x_test)
print(accuracy_score(y_test,pred_y))



In [ ]:
from sklearn.metrics import accuracy_score
clf2=RandomForestClassifier()
model2=clf2.fit(x_train,y_train)
pred_y=model2.predict(x_test)
print(accuracy_score(y_test,pred_y))

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
clf3=SVC()
model3=clf3.fit(x_train,y_train)
pred_y=model3.predict(x_test)
print(accuracy_score(y_test,pred_y))

In [ ]:
testdf=pd.read_csv("../input/amazon-ml-engineer-hiring-challenge/test.csv")

In [ ]:
testdf=testdf.dropna()
user_test=testdf.pop("customer_id")

In [ ]:
test_x=pd.get_dummies(testdf,columns=['customer_active_segment','X1'])

In [ ]:
predictions=model2.predict(test_x)
print(predictions)

In [ ]:
to_submit=pd.DataFrame(list(zip(user_test,predictions)),columns=['userid','customer_category'])